In [1]:
%load_ext autoreload
%autoreload 2

import sys
import copy
import math
import pickle

#sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import meshio

#import implicitmodules.torch as dm

In [3]:
pwd

'/home/gris/algos/implicitmodules/script/torch'

In [4]:
import os
os.chdir('/home/gris/algos/implicitmodules')

In [5]:
import implicitmodules.torch as dm
torch.set_default_tensor_type(torch.FloatTensor)

dm.Utilities.set_compute_backend('torch')

In [6]:
def set_aspect_equal_3d(ax):
    """Fix equal aspect bug for 3D plots."""

    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()

    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)

    plot_radius = max([abs(lim - mean_)
                       for lims, mean_ in ((xlim, xmean),
                                           (ylim, ymean),
                                           (zlim, zmean))
                       for lim in lims])

    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])

In [7]:
dim = 3

rmin_rigid = 1
rmax_rigid = 4.5
N_layerrigid = 4
rlist_rigid = torch.linspace(rmin_rigid, rmax_rigid, N_layerrigid)

rmin_growth = 5.2
rmax_growth = 6.
N_layergrowth = 2
rlist_growth = torch.linspace(rmin_growth, rmax_growth, N_layergrowth)


In [8]:
Nangle = 40
theta = torch.linspace(0., 2. * math.pi, Nangle)
gap = 0.5

R_list =  torch.stack([torch.cat([torch.cat([dm.Utilities.rot2d(a), torch.zeros([2,1])], dim=1), torch.tensor([[0., 0., 1.]])]) for a in theta])


In [9]:
pts1_toprigid = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), gap] for th in theta]) for r in rlist_rigid])
C_toprigid = torch.zeros([pts1_toprigid.shape[0], dim, 1])
R_toprigid = torch.cat([R_list.clone() for r in rlist_rigid])

pts1_midrigid = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), 0.] for th in theta]) for r in rlist_rigid])
C_midrigid = torch.zeros([pts1_toprigid.shape[0], dim, 1])
R_midrigid = torch.cat([R_list.clone() for r in rlist_rigid])

pts1_downrigid = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), -gap] for th in theta]) for r in rlist_rigid])
C_downrigid = torch.zeros([pts1_toprigid.shape[0], dim, 1])
R_downrigid = torch.cat([R_list.clone() for r in rlist_rigid])

In [10]:
Nb_ptslayerrigid = pts1_toprigid.shape[0]

In [11]:
k=4
costheta2 = torch.cat([torch.tensor([math.cos(k*th) for th in theta]), torch.tensor([math.cos(k * th) for th in theta])])

In [12]:
pts1_topgrowth = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), gap] for th in theta]) for r in rlist_growth])
C_topgrowth = torch.zeros([pts1_topgrowth.shape[0], dim, 1])
C_topgrowth[:, 1, :] = 1. + costheta2.view(-1, 1)
R_topgrowth = torch.cat([R_list.clone() for r in rlist_growth])

pts1_midgrowth = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), 0.] for th in theta]) for r in rlist_growth])
C_midgrowth = torch.zeros([pts1_topgrowth.shape[0], dim, 1])
C_midgrowth[:, 1, :] = 1.
R_midgrowth = torch.cat([R_list.clone() for r in rlist_growth])

pts1_downgrowth = torch.cat([torch.tensor([[r * math.cos(th), r * math.sin(th), -gap] for th in theta]) for r in rlist_growth])
C_downgrowth = torch.zeros([pts1_topgrowth.shape[0], dim, 1])
C_downgrowth[:, 1, :] = 1. - costheta2.view(-1, 1)
R_downgrowth = torch.cat([R_list.clone() for r in rlist_growth])

In [13]:
Nb_ptslayergrowth = pts1_topgrowth.shape[0]

In [14]:
pts_implicit1 = torch.cat([pts1_toprigid, pts1_midrigid, pts1_downrigid, pts1_topgrowth, pts1_midgrowth, pts1_downgrowth])
C = torch.cat([C_toprigid, C_midrigid, C_downrigid, C_topgrowth, C_midgrowth, C_downgrowth])
R = torch.cat([R_toprigid, R_midrigid, R_downrigid, R_topgrowth, R_midgrowth, R_downgrowth])

In [15]:
pts1_toprigid.shape

torch.Size([160, 3])

In [16]:
print(pts_implicit1.shape, C.shape, R.shape)

torch.Size([720, 3]) torch.Size([720, 3, 1]) torch.Size([720, 3, 3])


In [17]:
%matplotlib qt5
ax = plt.subplot()
plt.plot(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), '.')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1[:,:2], C[:,:2,:], R=R[:, :2, :2], c_index=0, alpha=0.1, scale=1, color='r')
plt.axis('equal')
plt.show()

In [18]:
%matplotlib qt5
ax = plt.subplot()
plt.plot(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), '.')
Nr = 3 * Nb_ptslayerrigid
N0 = 3 * Nb_ptslayerrigid + Nangle
dm.Utilities.plot_C_ellipse(ax, pts_implicit1[:Nr,:2], C[:Nr,:2,:], R=R[:Nr, :2, :2], c_index=0, alpha=0.1, scale=1, color='r')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1[Nr : Nr + 2*Nangle,:2], C[Nr : Nr + 2*Nangle,:2,:], R=R[Nr : Nr + 2*Nangle, :2, :2], c_index=0, alpha=0.1, scale=1, color='r')
nm = N0 + 7
nM = N0 + 12
plt.plot(pts_implicit1[nm : nM , 0].numpy(), pts_implicit1[nm : nM, 1].numpy(), 'x')
nm = N0 + 7 - Nangle
nM = N0 + 12 - Nangle
plt.plot(pts_implicit1[nm : nM , 0].numpy(), pts_implicit1[nm : nM, 1].numpy(), 'x')

plt.axis('equal')
plt.show()

In [19]:
Nr

480

In [20]:
%matplotlib qt5
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), pts_implicit1[:, 2].numpy(), marker='.', color='b')
N0 = 3 * Nb_ptslayerrigid + Nangle
N1 = 3 * Nb_ptslayerrigid + Nangle + Nb_ptslayergrowth
N2 = 3 * Nb_ptslayerrigid + Nangle + 2 * Nb_ptslayergrowth
plt.plot(pts_implicit1[N0:N0 + Nangle, 0].numpy(), pts_implicit1[N0:N0 + Nangle, 1].numpy(), pts_implicit1[N0:N0 + Nangle, 2].numpy(), 'b')
plt.plot(pts_implicit1[N1:N1 + Nangle, 0].numpy(), pts_implicit1[N1:N1 + Nangle, 1].numpy(), pts_implicit1[N1:N1 + Nangle, 2].numpy(), 'b')
plt.plot(pts_implicit1[N2:N2 + Nangle, 0].numpy(), pts_implicit1[N2:N2 + Nangle, 1].numpy(), pts_implicit1[N2:N2 + Nangle, 2].numpy(), 'b')

set_aspect_equal_3d(ax)
plt.show()

In [19]:
k


4

In [20]:
Nangle

40

In [21]:
int(Nangle/(k+1)) 

8

In [24]:
sigma1 = 2.
nu = 0.01
cotan_pts = torch.zeros_like(pts_implicit1)
nm = N0 + 7
nM = N0 + 12
cot = 3000
cotan_pts[nm : nM  , 2] = cot
nm = N0 + 7 - Nangle
nM = N0 + 12 - Nangle
cotan_pts[nm : nM  , 2] = cot
cotan_R = torch.zeros_like(R)
#implicit1 = dm.DeformationModules.ElasticOrder1.ImplicitModule1.build_and_fill(dim, pts_implicit1.shape[0], C, sigma1, nu, coeff=1., gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()),  cotan = (cotan_pts.view(-1).requires_grad_(), cotan_R.view(-1).requires_grad_()))

In [29]:
implicit1 = dm.DeformationModules.create_deformation_module('implicit_order_1', dim=3, nb_pts=pts_implicit1.shape[0], C=C, sigma=sigma1, nu=nu, coeff=1., gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))


In [30]:
intermediate_states, intermediate_controls = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([implicit1]), 8, 'euler', intermediates=True)

In [24]:
pwd

'/home/gris/projects_py3/implicitmodules'

**Save and/or load result**

In [ ]:
pickle.dump({'intermediate_states':intermediate_states, 'intermediate_controls':intermediate_controls, 'sigma1':sigma1, 'rmin_rigid':rmin_rigid, 'rmax_rigid':rmax_rigid, 'N_layerrigid':N_layerrigid, 'rmin_growth':rmin_growth, 'rmax_growth':rmax_growth, 'N_layergrowth':N_layergrowth, 'Nangle':Nangle, 'gap':gap}, open("../../Results/ImplicitModules/3D/DiskFolding/shoot4_8.pkl", 'wb'))

In [23]:
res = pickle.load(open("../../Results/ImplicitModules/3D/DiskFolding/shoot4_8.pkl", 'rb'))
intermediate_states = res['intermediate_states']

***See result***

In [31]:
pts = implicit1.manifold.gd[0].view(-1, dim).detach()
t=-1
pts = intermediate_states[t].gd[0][0].view(-1, dim).detach()
%matplotlib qt5
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.scatter(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), pts_implicit1[:, 2].numpy(), marker='.', color='b')
ax.scatter(pts[:, 0].numpy(), pts[:, 1].numpy(), pts[:, 2].numpy(), marker='x', color='r')
plt.plot(pts[N0:N0 + Nangle, 0].numpy(), pts[N0:N0 + Nangle, 1].numpy(), pts[N0:N0 + Nangle, 2].numpy(), 'b')
plt.plot(pts[N1:N1 + Nangle, 0].numpy(), pts[N1:N1 + Nangle, 1].numpy(), pts[N1:N1 + Nangle, 2].numpy(), 'b')
plt.plot(pts[N2:N2 + Nangle, 0].numpy(), pts[N2:N2 + Nangle, 1].numpy(), pts[N2:N2 + Nangle, 2].numpy(), 'b')

set_aspect_equal_3d(ax)
plt.show()

In [ ]:
N00 = 0
for i in range(3 * (N_layerrigid)):
    plt.plot(pts[N00:N00 + Nangle, 0].numpy(), pts[N00:N00 + Nangle, 1].numpy(), pts[N00:N00 + Nangle, 2].numpy(), 'k')
    N00 += Nangle
#N00 = 3 * N_layerrigid
for i in range(3 * (N_layergrowth)):
    plt.plot(pts[N00:N00 + Nangle, 0].numpy(), pts[N00:N00 + Nangle, 1].numpy(), pts[N00:N00 + Nangle, 2].numpy(), 'k')
    N00 += Nangle
N00 = 0
#for i in range(Nangle):
#    plt.plot(pts[N00:N0:Nangle+1, 0].numpy(), pts[N00:N0:Nangle+1, 1].numpy(), pts[N00:N0:Nangle+1, 2].numpy(), 'b')
#    N00 += N_layerrigid

N00 = 0
for j in range(3):
    N00 = j * Nb_ptslayerrigid
    for i in range(Nangle):
        plt.plot(pts[N00:N00 + Nb_ptslayerrigid:Nangle, 0].numpy(), pts[N00:N00 +Nb_ptslayerrigid:Nangle, 1].numpy(), pts[N00:N00 + Nb_ptslayerrigid:Nangle, 2].numpy(), 'k')
        N00 += 1
        
        
N00 = 0
for j in range(3):
    N00 = Nr + j * Nb_ptslayergrowth
    for i in range(Nangle):
        plt.plot(pts[N00:N00 + Nb_ptslayergrowth:Nangle, 0].numpy(), pts[N00:N00 +Nb_ptslayergrowth:Nangle, 1].numpy(), pts[N00:N00 + Nb_ptslayergrowth:Nangle, 2].numpy(), 'k')
        N00 += 1

N00 = Nr
for i in range(int(pts.shape[0]/3)):
        plt.plot(pts[N00::Nangle, 0].numpy(), pts[N00::Nangle, 1].numpy(), pts[N00::Nangle, 2].numpy(), 'k')
        N00 +=1

In [24]:
def save3Dplot(t, elev, azim, name):

    #pts = implicit1.manifold.gd[0].view(-1, dim).detach()
    #t= 0
    Nr = 3 * Nb_ptslayerrigid 
    #pts = intermediate_states[t].gd[0][0].view(-1, dim).detach()
    #%matplotlib qt5
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    #ax.scatter(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), pts_implicit1[:, 2].numpy(), marker='.', color='b')
    #ax.scatter(pts[:, 0].numpy(), pts[:, 1].numpy(), pts[:, 2].numpy(), marker='x', color='r')

    #t= 8
    Nr = 3 * Nb_ptslayerrigid 
    pts = intermediate_states[t].gd[0][0].view(-1, dim).detach()
    #%matplotlib qt5
    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.scatter(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), pts_implicit1[:, 2].numpy(), marker='.', color='b')
    #ax.scatter(pts[:, 0].numpy(), pts[:, 1].numpy(), pts[:, 2].numpy(), marker='x', color='r')
    N00 = 0
    for i in range(3 * (N_layerrigid)):
        plt.plot(pts[N00:N00 + Nangle, 0].numpy(), pts[N00:N00 + Nangle, 1].numpy(), pts[N00:N00 + Nangle, 2].numpy(), '-b')
        N00 += Nangle
    #N00 = 3 * N_layerrigid
    for i in range(3 * (N_layergrowth)):
        plt.plot(pts[N00:N00 + Nangle, 0].numpy(), pts[N00:N00 + Nangle, 1].numpy(), pts[N00:N00 + Nangle, 2].numpy(), '-r')
        N00 += Nangle
    N00 = 0
    #for i in range(Nangle):
    #    plt.plot(pts[N00:N0:Nangle+1, 0].numpy(), pts[N00:N0:Nangle+1, 1].numpy(), pts[N00:N0:Nangle+1, 2].numpy(), 'b')
    #    N00 += N_layerrigid

    N00 = 0
    for j in range(3):
        N00 = j * Nb_ptslayerrigid
        for i in range(Nangle):
            plt.plot(pts[N00:N00 + Nb_ptslayerrigid:Nangle, 0].numpy(), pts[N00:N00 +Nb_ptslayerrigid:Nangle, 1].numpy(), pts[N00:N00 + Nb_ptslayerrigid:Nangle, 2].numpy(), '-b')
            N00 += 1


    N00 = 0
    for j in range(3):
        N00 = Nr + j * Nb_ptslayergrowth
        for i in range(Nangle):
            plt.plot(pts[N00:N00 + Nb_ptslayergrowth:Nangle, 0].numpy(), pts[N00:N00 +Nb_ptslayergrowth:Nangle, 1].numpy(), pts[N00:N00 + Nb_ptslayergrowth:Nangle, 2].numpy(), '-r')
            N00 += 1


    N00 = Nr + Nangle
    for i in range(int(pts.shape[0]/3)):
            plt.plot(pts[N00::2*Nangle, 0].numpy(), pts[N00::2*Nangle, 1].numpy(), pts[N00::2*Nangle, 2].numpy(), 'r')
            N00 +=1



    set_aspect_equal_3d(ax)
    #ax.view_init(elev=20., azim=50)
    #ax.view_init(elev=40., azim=50)
    #ax.view_init(elev=40., azim=20)
    ax.view_init(elev=elev, azim=azim)
    plt.show()
    plt.savefig(name, format='pdf', bbox_inches = 'tight')
    plt.close('all')

In [25]:
t=0
elevlist = [0., 20., 40.]
azimlist = [0., 20, 50.]
for elev in elevlist:
    for azim in azimlist:
        for t in range(9):
            name = "../../Results/ImplicitModules/3D/DiskFolding/shoot4_8_t__" + str(t) + '_elev_' + str(int(elev)) + '_azim' + str(int(azim)) + '.pdf'
            save3Dplot(t, elev, azim, name)

In [25]:
%matplotlib qt5
name = "../../Results/ImplicitModules/3D/DiskFolding/shoot4_8__Ctop.pdf"
#fig=plt.figure()
ax = plt.subplot()
plt.plot(pts_implicit1[:, 0].numpy(), pts_implicit1[:, 1].numpy(), '.')
Nr = 3 * Nb_ptslayerrigid
N0 = 3 * Nb_ptslayerrigid + 2 * Nangle
N1 = 3 * Nb_ptslayerrigid + 4 *Nangle
dm.Utilities.plot_C_ellipse(ax, pts_implicit1[:Nr,:2], C[:Nr,:2,:], R=R[:Nr, :2, :2], c_index=0, alpha=0.1, scale=1, color='r')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1[Nr : Nr + 2*Nangle,:2], C[Nr : Nr + 2*Nangle,:2,:], R=R[Nr : Nr + 2*Nangle, :2, :2], c_index=0, scale=0.5, color='r')
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1[N0 : N1, :2], C[N0 : N1,:2,:], R=R[N0 : N1, :2, :2], c_index=0, scale=0.5, color='r')
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1[N1:,:2], C[N1:,:2,:], R=R[N1:, :2, :2], c_index=0, scale=0.5, color='r')
plt.axis('equal')
ax.set_title('Top layer')
plt.show()
#plt.savefig(name, format='pdf', bbox_inches = 'tight')
#plt.close('all')

In [26]:
len(intermediate_states)

9

In [55]:
Nangle * N_layerrigid

120

In [56]:
Nb_ptslayerrigid

120

In [26]:
pts.shape[0]

720